# Doest TradingView combined signals works? 


![title](img1.png)

## Data

In [2]:
library(tidyverse)
library(DT)

.av_api_key <- "your_key_here"

# Get data
data     <- read_csv("data.csv", col_types = cols())
products <- read_csv("products.csv", col_types = cols())

### Products

In [10]:
products %>% arrange(type, ticker)

ticker,type,name
<chr>,<chr>,<chr>
BNBUSD,crypto,Binance Coin / US Dollar (calculated by TradingView)
BTCUSD,crypto,Bitcoin / Dollar
DASHUSD,crypto,Dash / US Dollar (calculated by TradingView)
EOSUSD,crypto,EOS / US Dollar (calculated by TradingView)
ETHUSD,crypto,Ethereum / US Dollar (calculated by TradingView)
IOTAUSD,crypto,IOTA / US Dollar (calculated by TradingView)
LTCUSD,crypto,Litecoin / US Dollar (calculated by TradingView)
XRPUSD,crypto,XRP / U.S. dollar
DAX,index,DAX INDEX


In [11]:
data %>%
  group_by(type, ticker) %>%
  summarise(
    from_date = min(date), 
    to_date   = max(date), 
    days_cny  = n()
  ) 

type,ticker,from_date,to_date,days_cny
<chr>,<chr>,<date>,<date>,<int>
crypto,BNBUSD,2019-09-22,2020-02-22,154
crypto,BTCUSD,2019-09-22,2020-02-22,154
crypto,DASHUSD,2019-09-22,2020-02-22,154
crypto,EOSUSD,2019-09-22,2020-02-22,154
crypto,ETHUSD,2019-09-22,2020-02-22,154
crypto,IOTAUSD,2019-09-22,2020-02-22,154
crypto,LTCUSD,2019-09-22,2020-02-22,154
crypto,XRPUSD,2019-09-22,2020-02-22,154
index,DAX,2019-09-23,2020-02-21,105


### Signals

In [6]:
data %>% count(rating_osc)

rating_osc,n
<chr>,<int>
Buy,1136
Neutral,790
Sell,986


In [7]:
data %>% count(rating_ma)

rating_ma,n
<chr>,<int>
Buy,362
Neutral,78
Sell,223
Srong Buy,1511
Srong Sell,738


## Market data

In [17]:
# ETF used
etf_code <- c("SPX"   = "SPY", 
              "IXIC"  = "ONEQ", 
              "DJI"   = "DIA", 
              "VIX"   = "VXX", 
              "NI225" = 'CNKY.LON',
              "TSX"   = "XIU.TRT", 
              "DAX"   = "DAX", 
              "NIFTY" = "NFTY")

In [ ]:
data_list <- lapply(
  1:nrow(products), 
  function(i){
    Sys.sleep(12)
    print(i)
    
    if (products$type[i] == "crypto"){
      
      symbol <- str_replace(products$ticker[i], "USD", "")
      
      api_url <- str_c("https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=", symbol ,"&datatype=csv&market=USD&apikey=", .av_api_key )
      
      data <- read_csv(api_url)
      
      data <- data %>%
        mutate(
          date   = as.Date(timestamp),
          ticker = products$ticker[i]
        ) %>%
        filter(date >= '2019-01-01') %>%
        select(ticker, date, close = "close (USD)", )
      
      data
      
    } else {
      
      symbol <- if_else(products$type[i] == "index", etf_code[products$ticker[i]], products$ticker[i])
      
      api_url <- str_c("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=", symbol ,"&datatype=csv&outputsize=full&apikey=", .av_api_key)
      
      data <- read_csv(api_url)
      
      data <- data %>%
        mutate(
          date   = as.Date(timestamp),
          ticker = products$ticker[i]
        ) %>%
        filter(date >= '2019-01-01') %>%
        select(ticker, date, close)
      
      data
    }
  }
)

data_df <- bind_rows(data_list)

# Calculate daily P&L and add to the data 
data_df <- data_df %>%
  group_by(ticker) %>%
  arrange(date) %>%
  mutate(
    pnl = replace_na(close / lag(close) - 1, 0)
  )

data <- data %>%
  inner_join(data_df, by = c("date", "ticker"))


## Strategy

In [13]:
tibble(
  Rating = c('Strong Sell', 'Sell', 'Neutral', "Buy", "Srong Buy"), 
  Osc    = c(NA, "-100%", "0%", "100%", NA), 
  MA     = c("-100%", "-50%", "0%", "50%", "100%")
)

Rating,Osc,MA
<chr>,<chr>,<chr>
Strong Sell,NA,-100%
Sell,-100%,-50%
Neutral,0%,0%
Buy,100%,50%
Srong Buy,NA,100%


In [27]:
# Adjust P&L for Osc and MA
data <- data %>%
  mutate(
    pnl_osc = case_when(
      rating_osc == 'Buy'  ~ pnl,
      rating_osc == 'Sell' ~ -pnl,
      TRUE            ~ 0
    ), 
    pnl_ma = case_when(
      rating_osc == 'Buy'        ~  pnl / 2,
      rating_osc == 'Sell'       ~ -pnl / 2,
      rating_osc == 'Srong Buy'  ~  pnl,
      rating_osc == 'Srong Sell' ~ -pnl,
      TRUE            ~ 0
    )
  )

# Calculate Buy and Hold, Osc and MA returns
results <- data %>%
  group_by(type, ticker) %>%
  summarise(
    hold_result = round(prod(1 + pnl)     - 1, 4) * 100,
    osc_result  = round(prod(1 + pnl_osc) - 1, 4) * 100,
    ma_result   = round(prod(1 + pnl_ma)  - 1, 4) * 100,
    osc_better  = if_else(osc_result  > hold_result, 'x', ""),
    ma_better   = if_else(ma_result   > hold_result, 'x', "")
  )

## Results

In [28]:
results %>%
 mutate(
    hold_result = str_c(hold_result, "%"),
    osc_result  = str_c(osc_result,  "%"),
    ma_result   = str_c(ma_result,   "%")
 )

type,ticker,hold_result,osc_result,ma_result,osc_better,ma_better
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
crypto,BNBUSD,4.78%,10.75%,7.99%,x,x
crypto,BTCUSD,-3.36%,45.35%,21.93%,x,x
crypto,DASHUSD,11.6%,-64.2%,-34.17%,,
crypto,EOSUSD,2.54%,17.24%,11.59%,x,x
crypto,ETHUSD,21.63%,-11.33%,-3.93%,,
crypto,IOTAUSD,-8.03%,-12.84%,-5.07%,,x
crypto,LTCUSD,2.14%,-34.28%,-16.75%,,
crypto,XRPUSD,-5.16%,-48.45%,-26.84%,,
index,DAX,7.22%,-11.13%,-5.67%,,
